In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime

Домашняя работа
Наборы данных вида Transactions (несколько транзакций на одного клиента) трансформировать в таблицу, где cl_id будут уникальными (соответственно 4000 строк в train и 1000 строк в test
Для каждого cl_id будет уникальное целевое событие target_flag, а также уникальный канал привлечения клиента channel_type (клиент привлекается лишь однажды и с самого начала его записи присваивается значение канала привлечения)
При агрегации (pandas.DataFrame.groupby) по cl_id (или по связке cl_id, channel_type, target_flag) необходимо создавать производные фичи, идеи для таких фичей могут быть следующими:

общая сумма транзакций по каждой из trx_category
общая сумма транзакции по основным вылютам (напр. выделить рубли, доллары и евро - предположительно, это будут самые крупные категории)
общая сумма транзакций по категориям MCC кодов (например, выбрать основные/популярные MCC коды). ВНИМАНИ! Некоторые MCC коды из train могут быть не представлены в test. Про MCC коды в целом: http://www.banki.ru/wikibank/mcc-kod/; Справочник MCC кодов: https://mcc-codes.ru/code; Про некоторые категории кэшбека Росбанка: https://mcc-codes.ru/card/rosbank-sverkh-plus;
возможные агрегации по времени суток и дням недели - траты в выходные (праздники) или будни, в ночное время или в рабочее и т.д.

Обязательная часть: провести первичный анализ данных - посмотреть распределения признаков, выделить самые популярные MCC, помотреть активность клиентов по дням недели/времени, какие категории транзакции (trx_category) наиболее популярны и т.д. Получить инсайты, которые в дальнейшем помогут вам правильно подготовить фичи

In [50]:
raw_df = pd.read_csv('rosbank_train.csv')
cl_ids_test = np.random.choice(raw_df.cl_id.unique(), size=1000, replace=False)
cl_ids_test_set = set(cl_ids_test)
# create transactions dataset for train
transactions_train = raw_df[~raw_df.cl_id.isin(cl_ids_test)].copy()
print("Total transactions in train dataset: ", len(transactions_train))
# create transactions dataset for test
transactions_test = raw_df[raw_df.cl_id.isin(cl_ids_test)].copy()
print("Total transactions in test dataset: ", len(transactions_test))

Total transactions in train dataset:  393860
Total transactions in test dataset:  96653


In [3]:
transactions_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392150 entries, 109 to 490510
Data columns (total 10 columns):
PERIOD          392150 non-null object
cl_id           392150 non-null int64
MCC             392150 non-null int64
channel_type    389613 non-null object
currency        392150 non-null int64
TRDATETIME      392150 non-null object
amount          392150 non-null float64
trx_category    392150 non-null object
target_flag     392150 non-null int64
target_sum      392150 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 32.9+ MB


In [55]:
transactions_train.head(9)

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0,0.0
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0,0.0
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0,0.0
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0,0.0
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0,0.0
5,01/10/2017,1,5814,NaN,810,16OCT17:00:00:00,380.0,POS,0,0.0
6,01/10/2017,1,5814,NaN,810,10OCT17:00:00:00,378.0,POS,0,0.0
7,01/10/2017,1,5814,NaN,810,16OCT17:00:00:00,199.0,POS,0,0.0
8,01/10/2017,1,5814,NaN,810,11OCT17:00:00:00,400.0,POS,0,0.0


In [41]:
#распределение транзакции по основным вылютам для клиента

trx_bycl_bycurrency = pd.DataFrame(raw_df[
    ['cl_id','currency','amount']
    ].groupby(['cl_id','currency']).agg('sum',axis = 'columns')).reset_index()

trx_bycl_bycurrency.head(15)



,cl_id,currency,amount
0,0,810,64383.00
1,1,810,266693.13
2,1,978,884.91
3,5,504,1442.67
4,5,810,544874.63
5,5,978,51.83
6,9,810,849315.09
7,10,810,1124343.99
8,11,810,427662.85
9,11,978,84.29


In [19]:
#самые популярные виды транзакций
raw_df['trx_category'].value_counts()

POS               416425
DEPOSIT            21216
WD_ATM_ROS         19104
WD_ATM_PARTNER      9948
C2C_IN              7306
WD_ATM_OTHER        7140
C2C_OUT             5456
BACK_TRX            2687
CAT                 1197
CASH_ADV              34
Name: trx_category, dtype: int64

In [31]:
#самые популярные категории транзакций
df_mcc = raw_df['MCC'].value_counts()
df_mcc.head(15)

5411    121640
6011     54382
5814     41351
5812     30027
5499     27237
5541     19816
5912     18728
5999     13073
6012     10056
5921      8578
5331      7641
4121      6266
5211      6262
4829      6205
5691      5161
Name: MCC, dtype: int64

In [42]:
#распределение транзакции по типам платежей для клиента

trx_bycl_bymcc = pd.DataFrame(raw_df[
    ['cl_id','MCC','target_flag','amount']
    ].groupby(['cl_id','MCC','target_flag']).agg('sum',axis = 'columns')).reset_index()

trx_bycl_bymcc.head(15)


,cl_id,MCC,target_flag,amount
0,0,5200,0,5023.00
1,0,5411,0,2031.00
2,0,5921,0,767.00
3,0,6011,0,20000.00
4,0,6012,0,36562.00
5,1,4111,0,17.10
6,1,4468,0,34.00
7,1,5200,0,23.90
8,1,5411,0,14629.46
9,1,5499,0,492.80


In [51]:
#Добавим столбцы с информацией по времени транзакции и дне недели
raw_df['day_of_trx'] = raw_df['TRDATETIME'].apply(lambda x:(datetime.datetime.strptime(x,'%d%b%y:%H:%M:%S')).strftime('%w'))
raw_df['hour_of_trx'] = raw_df['TRDATETIME'].apply( lambda x: (datetime.datetime.strptime(x, '%d%b%y:%H:%M:%S')).strftime('%H') ) 
raw_df.head(15)



,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum,day_of_trx,hour_of_trx
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0,0.0,6,00
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0,0.0,4,12
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0,0.0,2,00
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0,0.0,6,00
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0,0.0,2,13
5,01/10/2017,1,5814,NaN,810,16OCT17:00:00:00,380.0,POS,0,0.0,1,00
6,01/10/2017,1,5814,NaN,810,10OCT17:00:00:00,378.0,POS,0,0.0,2,00
7,01/10/2017,1,5814,NaN,810,16OCT17:00:00:00,199.0,POS,0,0.0,1,00
8,01/10/2017,1,5814,NaN,810,11OCT17:00:00:00,400.0,POS,0,0.0,3,00
9,01/07/2017,1,5411,NaN,810,26JUL17:00:00:00,598.0,POS,0,0.0,3,00


In [53]:
#транзакции по времени совершения

raw_df['hour_of_trx'].value_counts()

00    432913
13      4843
12      4707
11      4681
14      4397
15      4120
10      4088
16      3848
17      3816
18      3659
09      3635
19      2704
08      2699
07      1983
20      1797
06      1315
21      1268
05       981
22       850
04       660
23       491
03       456
02       350
01       252
Name: hour_of_trx, dtype: int64

In [55]:
#транзакции по дням недели

raw_df['day_of_trx'].value_counts()

5    78993
6    74027
4    71458
3    69485
2    68509
1    65927
0    62114
Name: day_of_trx, dtype: int64